In [1]:
import pandas as pd

In [8]:
import json
from pandas.io.json import json_normalize    

with open('emp_1.json') as data_file:    
    data = json.load(data_file)  

mapping_data = json_normalize(data, 'ColumnNames', ['db', 'tableName', 'transfermation'],record_prefix='ColumnNames_')

mapping_data

,ColumnNames_datatype,ColumnNames_dest,ColumnNames_source,db,tableName,transfermation
0,int,dest_col1,col1,xxxxxxzzzz,abc_table,update
1,string,dest_col3,col2,xxxxxxzzzz,abc_table,update
2,double,dest_col2,col3,xxxxxxzzzz,abc_table,update


In [9]:
mapping_data[['ColumnNames_datatype','ColumnNames_dest','ColumnNames_source']]

,ColumnNames_datatype,ColumnNames_dest,ColumnNames_source
0,int,dest_col1,col1
1,string,dest_col3,col2
2,double,dest_col2,col3


In [21]:
# csv_df=spark.read.csv("file:///home/hduser/Desktop/Mandip/data.csv",header=True,inferSchema='true')

In [4]:
# csv_df.printSchema()

In [23]:
# csv_df.write.parquet("file:///home/hduser/Desktop/Mandip/data_par")

In [5]:
par_df=spark.read.parquet("file:///home/hduser/Desktop/Mandip/data_par")

In [6]:
par_df.printSchema()

root
 |-- col1: integer (nullable = true)
 |-- col2: string (nullable = true)
 |-- col3: double (nullable = true)
 |-- col4: string (nullable = true)



In [7]:
par_df.show()

+----+-------+-----+-------+
|col1|   col2| col3|   col4|
+----+-------+-----+-------+
|   1| value1|100.5|  data1|
|   2| value2|105.5| data10|
|   3| value3|110.5|data100|
|   4| value4|115.5|  data2|
|   5| value5|120.5| data11|
|   6| value6|125.5|data101|
|   7| value7|130.5|  data3|
|   8| value8|135.5| data12|
|   9| value9|140.5|data102|
|  10|value10|145.5|  data4|
|  11|value11|150.5| data13|
|  12|value12|155.5|data103|
|  13|value13|160.5|  data5|
|  14|value14|165.5| data14|
|  15|value15|170.5|data104|
|  16|value16|175.5|  data6|
|  17|value17|180.5| data15|
|  18|value18|185.5|data105|
|  19|value19|190.5|  data7|
|  20|value20|195.5| data16|
+----+-------+-----+-------+



In [14]:
from pyspark.sql.types import *
from pyspark.sql.types import StringType,IntegerType,DoubleType,StructField,StructType
field =[]
col_names_dest=list(mapping_data['ColumnNames_dest'])
col_names_source=list(mapping_data['ColumnNames_source'])
data_types=list(mapping_data['ColumnNames_datatype'])
for col_name,data_type in zip(col_names_dest,data_types):

    if data_type=="string":
        field.append(StructField(col_name,StringType()))
    elif data_type=="int":
        field.append(StructField(col_name,IntegerType()))
    elif data_type=="double":
        field.append(StructField(col_name,DoubleType()))
    else:
        print("Something went wrong with Data types")
        

schema = StructType(field)
dest_df= spark.createDataFrame(sc.emptyRDD(), schema)

In [15]:
dest_df.printSchema()

root
 |-- dest_col1: integer (nullable = true)
 |-- dest_col3: string (nullable = true)
 |-- dest_col2: double (nullable = true)



In [17]:
par_df.createOrReplaceTempView("par_view")

In [16]:
queryformation=""
for col_name_dest,col_name_source in zip(col_names_dest,col_names_source):
    queryformation=queryformation+col_name_source+" as "+col_name_dest+","


In [18]:
queryformation="select "+queryformation[0:-1]+" from par_view"


In [19]:
dest_df_2=spark.sql(queryformation)

In [20]:
dest_df_2.show()

+---------+---------+---------+
|dest_col1|dest_col3|dest_col2|
+---------+---------+---------+
|        1|   value1|    100.5|
|        2|   value2|    105.5|
|        3|   value3|    110.5|
|        4|   value4|    115.5|
|        5|   value5|    120.5|
|        6|   value6|    125.5|
|        7|   value7|    130.5|
|        8|   value8|    135.5|
|        9|   value9|    140.5|
|       10|  value10|    145.5|
|       11|  value11|    150.5|
|       12|  value12|    155.5|
|       13|  value13|    160.5|
|       14|  value14|    165.5|
|       15|  value15|    170.5|
|       16|  value16|    175.5|
|       17|  value17|    180.5|
|       18|  value18|    185.5|
|       19|  value19|    190.5|
|       20|  value20|    195.5|
+---------+---------+---------+



In [21]:
dest_df_2.printSchema()

root
 |-- dest_col1: integer (nullable = true)
 |-- dest_col3: string (nullable = true)
 |-- dest_col2: double (nullable = true)



In [22]:
dest_df_2.write.parquet("file:///home/hduser/Desktop/Mandip/data_par_new")